In [4]:
import math, pathlib, shutil, textwrap
# 计算 2D 单峰 von Mises 分布的 mu（均值方向），从 3D 旋转矩阵的前向向量投影得到
def wrap_to_pi(a: float) -> float:
    a = (a + math.pi) % (2*math.pi) - math.pi
    if a <= -math.pi:
        a += 2*math.pi
    return a

def parse_forward_vec(txt_path: pathlib.Path):
    xs = []
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line: continue
            xs.append([float(t) for t in line.split()])
    if len(xs) < 3 or len(xs[2]) < 3:
        raise ValueError(f"格式错误: {txt_path}")
    fx, fy, fz = xs[2]  # 第3行为前向
    # 投影到水平面并归一化
    r = math.hypot(fx, fz)     # sqrt(fx^2 + fz^2)
    if r < 1e-8:
        fx, fz = 0.0, -1.0     # 退化处理：当作正对 -z
    else:
        fx, fz = fx/r, fz/r
    return fx, fz

def vec_to_mu(fx: float, fz: float) -> float:
    # 标准极角(从 +x 逆时针)：atan2(y, x) => atan2(fz, fx)
    theta_std = math.atan2(fz, fx)      # (-pi, pi]
    # 平移，使 -z 成为 0°
    mu = wrap_to_pi(theta_std + math.pi/2)
    return mu
def vec_to_mu_2(fx: float, fz: float) -> float:
    # 直接用 atan2(-fx, -fz)，等价于上面
    mu = math.atan2(fx, -fz)           # (-pi, pi]
    return mu

root = pathlib.Path("./demo_vm_gt")
if root.exists(): shutil.rmtree(root)
root.mkdir(parents=True)

samples = {
    # 三行：侧向、竖直+Y、前向
    "case_A_baseline_-z.txt": textwrap.dedent("""\
        1 0 0
        0 1 0
        0 0 -1
    """),
    "case_B_forward_plus_x.txt": textwrap.dedent("""\
        0 0 1
        0 1 0
        1 0 0
    """),
    "case_C_random_rot.txt": textwrap.dedent("""\
        -0.662012 0.0  0.749493
         0.0      1.0  0.0
        -0.749493 0.0 -0.662012
    """),
    "case_D_near_vertical.txt": textwrap.dedent("""\
        1 0 0
        0 1 0
        0 1e-12 0     # 水平分量极小，触发退化到 -z
    """),
}

for name, content in samples.items():
    (root / name).write_text(content, encoding="utf-8")

# --------- 读取并计算 mu ---------
def deg(x): return x * 180.0 / math.pi

print(f"Demo dir: {root.resolve()}\n")
for p in sorted(root.glob("*.txt")):
    try:
        fx, fz = parse_forward_vec(p)
        mu = vec_to_mu(fx, fz)
        mu2 = vec_to_mu_2(fx, fz)
        print(mu,mu2)
        # 展示：标准极角与“以 -z 为 0°”的角度（度）
        theta_std = math.atan2(fz, fx)
        mu_deg = deg(mu)                    # (-180,180]
        mu_deg_0to360 = (mu_deg + 360) % 360
        print(f"[{p.name}]")
        print(f"  forward_proj = (fx', fz') = ({fx:+.6f}, {fz:+.6f})  |  r=1.0")
        print(f"  theta_std(从 +x) = {theta_std:+.6f} rad  ({deg(theta_std):+.2f}°)")
        print(f"  mu(以 -z 为 0°) = {mu:+.6f} rad  ({mu_deg:+.2f}°)  ~ ({mu_deg_0to360:.2f}° in [0,360))")
        print()
    except Exception as e:
        print(f"[WARN] {p.name}: {e}")


Demo dir: /home/pablo/ForwardNet/data_process/demo_vm_gt

0.0 0.0
[case_A_baseline_-z.txt]
  forward_proj = (fx', fz') = (+0.000000, -1.000000)  |  r=1.0
  theta_std(从 +x) = -1.570796 rad  (-90.00°)
  mu(以 -z 为 0°) = +0.000000 rad  (+0.00°)  ~ (0.00° in [0,360))

1.5707963267948966 1.5707963267948966
[case_B_forward_plus_x.txt]
  forward_proj = (fx', fz') = (+1.000000, +0.000000)  |  r=1.0
  theta_std(从 +x) = +0.000000 rad  (+0.00°)
  mu(以 -z 为 0°) = +1.570796 rad  (+90.00°)  ~ (90.00° in [0,360))

-0.8472961006166244 -0.8472961006166242
[case_C_random_rot.txt]
  forward_proj = (fx', fz') = (-0.749493, -0.662012)  |  r=1.0
  theta_std(从 +x) = -2.418092 rad  (-138.55°)
  mu(以 -z 为 0°) = -0.847296 rad  (-48.55°)  ~ (311.45° in [0,360))

[WARN] case_D_near_vertical.txt: could not convert string to float: '#'


In [1]:
import math, pathlib, shutil, textwrap


def vec_to_mu(fx: float, fz: float) -> float:
    mu = math.atan2(fx, -fz)           # (-pi, pi]
    return mu


samples = [
    [1,0,-1],
    [-0.749493, 0.0, -0.662012],
    [0,1e-12,0],
]

def deg(x): 
    return x * 180.0 / math.pi,(x * 180.0 / math.pi + 360) % 360

for p in samples:
    try:
        mu = vec_to_mu(p[0], p[2])
        mu_deg,mu_deg_0to360 = deg(mu)   
        print(f"forward_proj fx' = {p[0]:+.6f}, fz' = {p[2]:+.6f}")    
        print(f"mu (-z 0°) {mu:+.6f} rad {mu_deg:+.2f}° in [-180,180] {mu_deg_0to360:.2f}° in [0,360))")
        print()
    except Exception as e:
        print(f"[WARN] {p.name}: {e}")

print(deg(-0.84729610))

forward_proj fx' = +1.000000, fz' = -1.000000
mu (-z 0°) +0.785398 rad +45.00° in [-180,180] 45.00° in [0,360))

forward_proj fx' = -0.749493, fz' = -0.662012
mu (-z 0°) -0.847296 rad -48.55° in [-180,180] 311.45° in [0,360))

forward_proj fx' = +0.000000, fz' = +0.000000
mu (-z 0°) +0.000000 rad +0.00° in [-180,180] 0.00° in [0,360))

(-48.54649052789455, 311.45350947210545)
